# Value Iteration Algorithms

In [1]:
import random
import numpy as np

In [2]:
# CONSTANTS
team_number = 92
arr = [1/2, 1, 2]
y = arr[team_number%3]
STEP_COST = -10/y

GAMMA = 0.999
DELTA = 0.001

In [3]:
X = 0
Y = 0
POSITION = "CENTER"

In [4]:
positions = {
    (0,0): "CENTER",
    (-1,0): "WEST",
    (1,0): "EAST",
    (0,1): "NORTH",
    (0,-1): "SOUTH"
}

coordinates = {
    "CENTER": (0,0),
    "WEST": (-1,0),
    "EAST": (1,0),
    "NORTH": (0,1),
    "SOUTH": (0,-1)
}

possible_actions = {
    "CENTER": ["UP", "DOWN", "LEFT", "RIGHT", "STAY", "SHOOT", "HIT"],
    "WEST": ["RIGHT", "STAY", "SHOOT"],
    "EAST": ["LEFT", "STAY", "SHOOT", "HIT"],
    "NORTH": ["DOWN", "STAY", "CRAFT"],
    "SOUTH": ["UP", "STAY", "GATHER"]
}
direction_tuple = ["CENTER", "WEST", "EAST", "NORTH", "SOUTH"]

In [5]:
utilities = np.zeros((5,3,4,2,5))
history = []
action_hist = []
history.append(utilities)
# location, materials, arrows, monster attacc?, monster aliven'tn't

In [6]:
# MOVEMENT FUNCTIONS

def move_up(state):
    new_state = state.copy()
    
    old_coordinates = coordinates[direction_tuple[new_state[0]]]
    (x, y) = old_coordinates
    y += 1
    coord = (x, y)
    new_state[0] = direction_tuple.index(positions[coord])
    
    return new_state
    

def move_down(pos):
    new_state = state.copy()
    
    old_coordinates = coordinates[direction_tuple[new_state[0]]]
    (x, y) = old_coordinates
    y -= 1
    coord = (x, y)
    new_state[0] = direction_tuple.index(positions[coord])
    return new_state
    

def move_left(pos):
    new_state = state.copy()
    
    old_coordinates = coordinates[direction_tuple[new_state[0]]]
    (x, y) = old_coordinates
    x -= 1
    coord = (x, y)
    new_state[0] = direction_tuple.index(positions[coord])
    return new_state
    

def move_right(pos):
    new_state = state.copy()
    
    old_coordinates = coordinates[direction_tuple[new_state[0]]]
    (x, y) = old_coordinates
    x += 1
    coord = (x, y)
    new_state[0] = direction_tuple.index(positions[coord])
    return new_state
    

def move_stay(pos):
    new_state = state.copy()
    return new_state
    

def move(action, state):
    switcher = {
        "UP": move_up,
        "DOWN": move_down,
        "LEFT": move_left,
        "RIGHT": move_right,
        "STAY": move_stay
    }
    
    return(switcher[action](state))

# move("UP", "hehe")

# print(Y)

In [7]:
## TRANSITION FUNCTIONS
class State:
    def __init__ (self, name, x, y, move_prob, shoot_prob = 0, craft_prob = 0, gather_prob = 0, hit_prob = 0):
        super().__init__()
        
        self.name = name
        self.x = x
        self.y = y
        self.move_prob = move_prob
        self.shoot_prob = shoot_prob
        self.shoot_damage = 25
        self.hit_prob = hit_prob
        self.hit_damage = 50
        self.craft_prob = craft_prob

In [8]:
## INDIANA CLASS
class Indiana:
    def __init__ (self):
        self.x = 0
        self.y = 0
        self.position = positions[(self.x, self.y)]
        self.arrows = 0
        self.material = 0
        self.health = 0
        
        self.north = State("NORTH", 0, 1, 0.85, craft_prob = {1: 0.5, 2: 0.35, 3: 0.15})
        self.south = State("SOUTH", 0, -1, 0.85, gather_prob = 0.75)
        self.east = State("EAST", 1, 0, 1, shoot_prob = 0.9, hit_prob = 0.2)
        self.west = State("WEST", 0, 1, 1, shoot_prob = 0.25)
        self.center = State("CENTER", 0, 0, 0.85, shoot_prob = 0.5, hit_prob = 0.1)
        
    def act(self):
        self.position = positions[(self.x, self.y)]
        
        cur_actions = possible_actions[self.position]
        
        best_action = cur_actions[0]
        best_utility = -999999
        
        this_utility = 0
        
        
#         for i in cur_actions:
            
            
        
    def move(self, direction):
        
        states
        prob = random.random()
#         if probability_movement[self.position] == "NORTH":
#             if prob <= 0.85:
                

In [9]:
def fml(state, action):
    location = direction_tuple[state[0]]
    
    ret = []
    
    if location == "NORTH":
        if action in ["DOWN", "STAY"]:
            new_state = move(action, state.copy())
            fail_state = state.copy()
            fail_state[0] = 2
            ret = [[0.85, 0.15], [new_state, fail_state]]
        else:
            arrow_probs = [0.5, 0.35, 0.15]
            arrow1 = state.copy()
            arrow2 = state.copy()
            arrow3 = state.copy()
            arrow1[1] -= 1
            arrow2[1] -= 1
            arrow3[1] -= 1
            arrow1[2] = min(3, arrow1[2] + 1)
            arrow2[2] = min(3, arrow2[2] + 2)
            arrow3[2] = min(3, arrow3[2] + 3)
            
            ret = [arrow_probs, [arrow1, arrow2, arrow3]]
            
  
    elif location == "SOUTH":
        if action in ["UP", "STAY"]:
            new_state = move(action, state.copy())
            fail_state = state.copy()
            fail_state[0] = 2
            ret = [[0.85, 0.15], [new_state, fail_state]]
        else:
            gather_probs = [0.75, 0.25]
            new_state = state.copy()
            fail_state = new_state.copy()
            new_state[1] = min(2, new_state[1] + 1)
            ret = [gather_probs, [new_state, fail_state]]
                
                
    elif location == "WEST":
        if action in ["RIGHT", "STAY"]:
            new_state = move(action, state.copy())
            fail_state = state.copy()
            fail_state[0] = 2
            ret = [[1, 0], [new_state, fail_state]]
        else:
            shoot_probs = [0.25, 0.75]
            new_state = state.copy()
            new_state[2] -= 1
            fail_state = new_state.copy()
            new_state[4] -= 1
            ret = [shoot_probs, [new_state, fail_state]]
            
            
    elif location == "EAST":
        if action in ["LEFT", "STAY"]:
            new_state = move(action, state.copy())
            fail_state = state.copy()
            fail_state[0] = 2
            ret = [[1, 0], [new_state, fail_state]]
        elif action == "SHOOT":
            shoot_probs = [0.9, 0.1]
            new_state = state.copy()
            new_state[2] -= 1
            fail_state = new_state.copy()
            new_state[4] -= 1
            ret = [shoot_probs, [new_state, fail_state]]
        else:
            hit_probs = [0.2, 0.8]
            new_state = state.copy()
            fail_state = new_state.copy()
            new_state[4] = max(0, new_state[4] - 2)
            ret = [hit_probs, [new_state, fail_state]]
            
            
    elif location == "CENTER":
        if action in ["UP", "DOWN", "RIGHT", "LEFT", "STAY"]:
            new_state = move(action, state.copy())
            fail_state = state.copy()
            fail_state[0] = 2
            ret = [[0.85, 0.15], [new_state, fail_state]]
        elif action == "SHOOT":
            shoot_probs = [0.5, 0.5]
            new_state = state.copy()
            new_state[2] -= 1
            fail_state = new_state.copy()
            new_state[4] -= 1
            ret = [shoot_probs, [new_state, fail_state]]
        else:
            hit_probs = [0.1, 0.9]
            new_state = state.copy()
            fail_state = new_state.copy()
            new_state[4] = max(0, new_state[4] - 2)
            ret = [hit_probs, [new_state, fail_state]]
            
    # monster slep slep 
    if not state[3]:
        awake_prob = [0.2, 0.8]
        probabilities = ret[0]
        states = ret[1]
        
        new_probs = []
        new_states = []
        
        for i, prob in enumerate(probabilities):
            new_probs.append(prob*awake_prob[0])
            new_probs.append(prob*awake_prob[1])
            awake_state = states[i].copy()
            awake_state[3] = 1
            new_states.append(awake_state)
            new_states.append(states[i].copy())
        
        ret = [new_probs, new_states]

    # monster wakey wakey   
    else:
        probabilities = ret[0]
        states = ret[1]
        
        new_probs = []
        new_states = []
        
        if location == "EAST" or "CENTER":
            attacc_prob = [0.5, 0.5]
            
            for i, prob in enumerate(probabilities):
                new_probs.append(prob*attacc_prob[0])
                new_states.append(states[i].copy())
            new_probs.append(attacc_prob[1])
            
            og_state = state.copy()
            og_state[2] = 0 # arrows
            og_state[4] = min(4, og_state[4] + 1) # helth
            og_state[3] = 0 # dormant state
            new_states.append(og_state)
            
        else:
            awake_prob = [0.5, 0.5]
            probabilities = ret[0]
            states = ret[1]

            for i, prob in enumerate(probabilities):
                new_probs.append(prob*awake_prob[0])
                new_probs.append(prob*awake_prob[1])
                awake_state = states[i].copy()
                awake_state[3] = 0
                new_states.append(awake_state)
                new_states.append(states[i].copy())
        ret = [new_probs, new_states]
            
    return ret

In [10]:
def reward(old, new):
    reward = STEP_COST
    if new[4] == 0:
        reward += 50
    elif direction_tuple[old[0]] in ["CENTER", "EAST"] and old[3] == 1 and new[3] == 0:
        reward -= 40
        
    return reward

In [12]:
## iteration loop

indiana = Indiana()
cur_error = 999999999

count = 0

while(cur_error > DELTA):
    print(count)
    this_utilities = np.zeros((5,3,4,2,5))


    for state, val in np.ndenumerate(utilities):

        state = list(state)

        cur_actions = possible_actions[direction_tuple[state[0]]].copy()

        if 'SHOOT' in cur_actions and state[2] == 0:
            cur_actions.remove('SHOOT')


        if 'CRAFT' in cur_actions and state[1] == 0:
            cur_actions.remove('CRAFT')

        if state[4] == 0:
            cur_actions = ['NONE']

        action_utils = []



        for action in cur_actions:
            utility = history[-1][tuple(state)]        

            if action != 'NONE':
                probs, states = fml(state, action)
                utility = 0

                for i, prob in enumerate(probs):
                    this_state = states[i].copy()
                    utility += prob*(reward(state, this_state) + GAMMA*(history[-1][tuple(this_state)]))


            action_utils.append(utility)
    #         print("action: ", action)
    #         print("utility: ", utility)


    #     print(action_utils)
        max_util = max(action_utils)
        gg_idx = action_utils.index(max_util)
        best_action = cur_actions[gg_idx]
#         print("BEST: ", best_action)
#         print("UTILITY: ", max_util)
        this_utilities[tuple(state)] = max_util

    # cur_error = 
    history.append(this_utilities)
    
    cur_error = np.max(np.abs(history[-1] - history[-2]))
    print(cur_error)
    
    count += 1


    
#     print()
        

0
40.00000000000001
1
18.32165000000001
2
11.122114252500007
3
7.100156857378494
4
6.506519203891429
5
6.384867530154949
6
6.26217692148272
7
6.124946946745631
8
5.943516384854732
9
5.760441876705919
10
5.278247006628767
11
4.9452741766477
12
4.940328902471059
13
4.935388573568588
14
4.930453184995017
15
4.92552273181002
16
4.92059720907821
17
4.915676611869131
18
4.910760935257272
19
4.905850174321998
20
4.900944324147673
21
4.531849965024904
22
3.513951216835366
23
2.8950994765241234
24
2.485581143133203
25
2.170358168356671
26
1.9138648731805006
27
1.714890423494296
28
1.543400802679443
29
1.3908274248226178
30
1.254420287354037
31
1.139876980029726
32
1.0405115340255975
33
0.9507285603901892
34
0.8692265492826436
35
0.7951932391171681
36
0.727979563537474
37
0.666874187603014
38
0.6114977099054499
39
0.5608737227303777
40
0.5145447266337158
41
0.47212199544199507
42
0.4332600883387272
43
0.39764651650912697
44
0.3649950002683795
45
0.3350507668659759
46
0.3075828150402913
47
0.2823

In [ ]:
print(possible_actions)